## **Introduction to DARLi, Basics of Backends and Models** 


The DARLi (Differentiable Articulated Robotics Library) is a wrapper around the CasADi kinematics/dynamics and Pinocchio libraries. Its primary goal is to facilitate the creation of differentiable models for robotic systems, provided a URDF (Unified Robot Description Format) file. **DARLi is not an implementation of mechanics-oriented** algorithms, such as Featherstone's Articulated Body or the Recursive Newton-Euler. Instead, we rely on the efficient implementations provided by [Pinocchio](https://github.com/stack-of-tasks/pinocchio/tree/master) and offer a wrapper that grants easy access to a suite of features that we and our colleagues have found useful in practice of trajectory planning, feedback control, and system identification over poly-articulated robots.

We will use [robot_descriptions.py](https://github.com/robot-descriptions/robot_descriptions.py) library to simplify the process of urdf retrieval.

**Note**: You have to restart kernel after `robot_descriptions` is installed

In [1]:

%%capture
!pip3 install robot_descriptions
from robot_descriptions import iiwa_description

### Backends 
The typical DARLi model is based on instance of Backend. Backend take `urdf` file as input and defines the way of computing two main functions: Articulated Bpdy Algorithm `aba` and Recursive Newton Euler `rnea`, as well as severals utility functions. These are soledly based on Pinnochio. For now there are two backends are available:

TODO: WHAT IS RNEA AND ABA

- `PinocchioBackend` - Provide interface to the Pinnochio python bindings. And may be useufull if one wants to obtain fast **numericall** computations. 
- `CasadiBackend` - The CasADi wrapper around Pinnochiio, this is used to cimpute dynamics in **symbolical** fashion, this allows for seeamless calculation of derivative and potentially used in trajectory optimization and optimal control.


The acces to backends is as simple as:

In [2]:
from darli.backend import CasadiBackend, PinocchioBackend

# Building the Pinnochio backend
pin_backend = PinocchioBackend(iiwa_description.URDF_PATH)
# Building the CasADi backend
cas_backend = CasadiBackend(iiwa_description.URDF_PATH)

In [3]:
pin_backend.nq
pin_backend.nv
pin_backend.aba
pin_backend.rnea

<bound method PinocchioBackend.rnea of <darli.backend._pinocchio.PinocchioBackend object at 0x7ff56828dc60>>

### Floating Base, Fixed joints and Conventions: 



TODO: Something on floating base and conventions

In [4]:
from robot_descriptions import go2_description

In [5]:
go2_cas = CasadiBackend(go2_description.URDF_PATH)

In [6]:
print(f"Dimensions of model, nq: {go2_cas.nq} nv: {go2_cas.nv}")

Dimensions of model, nq: 12 nv: 12


In [7]:
from darli.backend import JointType

go2_cas_floating = CasadiBackend(
    go2_description.URDF_PATH, root_joint=JointType.FREE_FLYER
)

print(
    f"Dimensions of floating model, nq: {go2_cas_floating.nq} nv: {go2_cas_floating.nv}"
)

Dimensions of floating model, nq: 19 nv: 18


In [8]:
cas_backend = CasadiBackend(go2_description.URDF_PATH, root_joint=JointType.FREE_FLYER)

The model in DARLi is a building block that can be further extended with external forces and contacts, different selector matrices, friction in joints, state space and parametric representations, please follow up to the next steps of tutorial to get in to that.

- General-info
- Backends
- Models 
- Floating Base,Conventions 
- Further Read and Structure


- Model of Poly-Articulated system
- Conventions
- rnea and aba
- Floating Base


### Conventions



### Backends

In [11]:
from darli.model import Model

In [13]:
casadi_model = Model(CasadiBackend(iiwa_description.URDF_PATH))

In [15]:
casadi_model.com()

CoM(position=SX(@1=cos(q_0), @2=6.16298e-32, @3=sin(q_1), @4=cos(q_1), @5=sin(q_2), @6=cos(q_2), @7=cos(q_3), @8=-1.22465e-16, @9=sin(q_4), @10=cos(q_4), @11=-1.47911e-33, @12=cos(q_5), @13=0.12228, @14=sin(q_5), @15=(0.00035+((@11*@12)-(@13*@14))), @16=-4.44089e-16, @17=(0.0735+((@11*(@16*@14))+((@13*(@16*@12))+-0.00072))), @18=(0.266+(((@11*@14)+((@13*@12)+-3.19744e-19))+0.6465)), @19=((((@8*@9)-@10)*@15)+(((@9+(@8*@10))*@17)+(-6.16298e-32*@18))), @20=sin(q_3), @21=(0.2345+((((@16*@9)*@15)+(((@16*@10)*@17)+@18))+1.19925)), @22=((@7*@19)-(@20*@21)), @23=-1.22465e-16, @24=4.44089e-16, @25=(0.119+((((@23*@10)+@9)*@15)+(((@10-(@23*@9))*@17)+(@24*@18)))), @26=(0.105+(((@16*@20)*@19)+(((@16*@7)*@21)-@25))), @27=1.22465e-16, @28=(0.455+(((@20*@19)+((@7*@21)+(@16*@25)))+2.155)), @29=(0.001905+((((@2*@5)-@6)*@22)+(((@5+(@2*@6))*@26)+(@27*@28)))), @30=1.22465e-16, @31=(0.37465+(((((@30*@6)+(@24*@5))*@22)+((((@24*@6)-(@30*@5))*@26)+@28))+2.76075)), @32=(0.2667+((@5*@22)+((@6*@26)+(@16*@28)))), 

One can retrive basic model info as follows:

In [ ]:
nq = casadi_model.nq  # dimensionality of configuration
nv = casadi_model.nv  # dimensionality of generilized velocities
nu = casadi_model.nu  # dimensionality  of control inputs
q_min, q_max = (
    casadi_model.q_min,
    casadi_model.q_max,
)  # minimal and maximal limits on joint pos
nq, nv, nu

(6, 6, 6)

In [ ]:
joint_names = casadi_model.joint_names  # names of the joints
joint_names

['universe', 'joint1', 'joint2', 'joint3', 'joint4', 'joint5', 'joint6']

### **Equations of Motion and Dynamics**

The dynamics of articulated mechanics in robotic systems is usually represented as:
$$
\mathbf{M}(\mathbf{q})\dot{\mathbf{v}} + \mathbf{C}(\mathbf{q},\mathbf{v})\mathbf{v} + \mathbf{g}(\mathbf{q})  = 
\mathbf{M}(\mathbf{q})\dot{\mathbf{v}} + \mathbf{c}(\mathbf{q},\mathbf{v}) + \mathbf{g}(\mathbf{q}) = \mathbf{M}(\mathbf{q})\dot{\mathbf{v}} + \mathbf{h}(\mathbf{q},\mathbf{v}) = \mathbf{Q}
$$

where:
* $\mathbf{Q} \in \mathbb{R}^{nv}$ - generalized forces corresponding to generilized coordinates
* $\mathbf{q} \in \mathbb{R}^{nq}$ - vector of generilized coordinates
* $\mathbf{v} \in \mathbb{R}^{nq}$ - vector of generilized velocities (sometimes $\mathbf{v} = \dot{\mathbf{q}}$, but not in case of $\mathbf{q}$ containing quaternions)
* $\mathbf{M} \in \mathbb{R}^{nv \times nv}$ - positive definite symmetric inertia matrix 
* $\mathbf{c} \in \mathbb{R}^{nv}$ - describe centrifugal and Coriolis forces
* $\mathbf{C} \in \mathbb{R}^{nv \times nv}$ - describe 'coefficients' of centrifugal and Coriolis forces
* $\mathbf{g} \in \mathbb{R}^{nv}$ - describes effect of gravity and other position depending forces
* $\mathbf{h} \in \mathbb{R}^{nv} $ - combined effect of $\mathbf{g}$ and $\mathbf{c}$


One can get all of the above quantities in symbotics as follows:

In [ ]:
inertia = casadi_model.inertia
gravity_vector = casadi_model.gravity
bias_force = casadi_model.bias_force
coriolis_matrix = casadi_model.coriolis_matrix
coriolis = casadi_model.coriolis

Each of the above define the CasAdi functions:

And can be evaluated both numerically and Functionalally:

In [ ]:
inertia = casadi_model.inertia
import numpy as np
import casadi as cs

# Functionalal computation
print("Functional:", inertia(cs.SX.sym("q", nq)))
# Numerical computation
print("Numerical:", inertia(np.random.randn(nq)))

Note that above are calculated not via Lagrange formalism but by using efficient recursive Newton-Euler algorithm (for `bias_force`), while inertia matrix is evaluated by composite rigid body algorithm (CRBA)

There are some notions apart from dynamical coefficients that may be useful as well, such as com kinematics and jacobians, and energy:

In [ ]:
model.energy

Energy(kinetic=Function(kinetic_energy:(q[6],v[6])->(kinetic_energy) SXFunction), potential=Function(potential_energy:(q[6])->(potential_energy) SXFunction))

In [ ]:
com_position = model.com.position
com_velocity = model.com.velocity
com_jacobian = model.com.jacobian
com_jacobian_dt = model.com.jacobian_dt
potential_energy = model.energy.potential
kinetic_energy = model.energy.kinetic
lagrangian = model.lagrangian

# TODO
# momentum

In [ ]:
com_jacobian_dt

Function(com_jacobian_dt:(q[6],v[6])->(com_jacobian_dt[3x6]) SXFunction)

#### **Forward and Inverse Dynamics**


In inverse dynamics we looking for the generilized forces:
$$
    \mathbf{Q} = \mathbf{M}(\mathbf{q})\dot{\mathbf{v}} + \mathbf{h}(\mathbf{q},\mathbf{v}) = \text{rnea}(\mathbf{q}, \mathbf{v}, \dot{\mathbf{v}})
$$

In [ ]:
model.contact_qforce

Function(contact_qforce:(q[6])->(contact_qforce[6]) SXFunction)

In [ ]:
model.inverse_dynamics

Function(inverse_dynamics:(q[6],v[6],dv[6])->(tau[6]) SXFunction)

While forward dynamics is basically solving the equations of motion with respect to accelerations $\dot{\mathbf{v}}$. However the solution of the above in general case usually yields the complexity of $O(nv^3)$, for this reason a way to go is to use celebrated Featherstone Articulated Body algorithm (ABA), which effectevely exploit sparsity of $\mathbf{M}$ for the tree structures and produce nearly linear complexity $O(nv)$:


$$
\dot{\mathbf{v}} = \mathbf{M}^{-1}(\mathbf{q})(\mathbf{Q} - \mathbf{h}(\mathbf{q},\mathbf{v})) = \text{aba}(\mathbf{q}, \mathbf{v}, \mathbf{Q})
$$

<!-- Articulated body algorithm 

Feather stone algorithm
 -->
<!-- http://gamma.cs.unc.edu/courses/robotics-f08/LEC/ArticulatedBodyDynamics.pdf -->

In [ ]:
model.contact_qforce

Function(contact_qforce:(q[6])->(contact_qforce[6]) SXFunction)

In [ ]:
model.forward_dynamics

Function(forward_dynamics:(q[6],v[6],tau[6])->(dv[6]) SXFunction)

#### **Passive Joints and Selector**

In practical situations 

Choosing the passive joints:

In [ ]:
import numpy as np

print(f"Old input dimensions: {model.nu}")
S = np.random.randn(model.nv, model.nv + 2)
model.update_selector(S)
print(f"New input dimensions: {model.nu}")

Old input dimensions: 6
New input dimensions: 8


In [ ]:
model.qfrc_u

SX([tau_0, tau_1, tau_2, tau_3, tau_4, tau_5, tau_6, tau_7])

In [ ]:
model.forward_dynamics

NameError: name 'model' is not defined

In [ ]:
model.update_selector(passive_joints=range(3))
print(f"New input dimensions: {model.nu}")
print(f"New selector:\n {model.selector}")

New input dimensions: 3
New selector:
 [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [ ]:
model.forward_dynamics

Function(forward_dynamics:(q[6],v[6],tau[3])->(dv[6]) SXFunction)

#### **Bodies and Contacts**

In [ ]:
model.add_body(["link06"])
model.bodies

{'link06': <darli.modeling.body.Body at 0x7f6ca40b11e0>}

In [ ]:
model.body("link06")

### Frames

In [ ]:
model.body("link06").linear_acceleration

FrameQuantity(local=Function(linear_acceleration_local:(i0[6],i1[6],i2[6])->(o0[3]) SXFunction), world=Function(linear_acceleration_world:(i0[6],i1[6],i2[6])->(o0[3]) SXFunction), world_aligned=Function(linear_acceleration_world_aligned:(i0[6],i1[6],i2[6])->(o0[3]) SXFunction))

In [ ]:
model.body("link06").linear_acceleration.local

Function(linear_acceleration_local:(i0[6],i1[6],i2[6])->(o0[3]) SXFunction)

In [ ]:
model.body("link06").jacobian.local

Function(jacobian_local:(i0[6])->(o0[6x6]) SXFunction)

In [ ]:
model.body("link03").jacobian.local
model.body("link03").jacobian_dt.local
model.body("link03").linear_velocity.local
model.body("link03").angular_velocity.local
model.body("link03").linear_acceleration.local
model.body("link03").angular_acceleration.local

Function(angular_acceleration_local:(i0[6],i1[6],i2[6])->(o0[3]) SXFunction)

The body jacobian and velocities can be calculated with respect to `world`, `local` and `world_aligned` frames.

In [ ]:
model.body("link06").jacobian.world

Function(jacobian_world:(i0[6])->(o0[6x6]) SXFunction)

Note that body name can be initialized with dictionary that maps given name to one presented in urdf i.e: `{'ee':'link06'}`

##### **Contacts**

In [ ]:
model.body("link06").add_contact("wrench")

In [ ]:
model.body("link06").contact.dim
# model.body("link06").contact.contact_frame
model.body("link06").contact.ref_frame
model.body("link06").contact.qforce

Function(qforce:(i0[6],i1[6])->(o0[6]) SXFunction)

Do not forget to rebuild the model:

In [ ]:
# model.update_model()

Note how arguments are changed in dynamics related functions, i.e:

In [ ]:
model.forward_dynamics

Function(forward_dynamics:(q[6],v[6],tau[3],link06[6])->(dv[6]) SXFunction)

the state space representation and jacobians are changed as well:

In [ ]:
model.state_space.state_derivative

Function(state_derivative:(q[6],v[6],tau[3],link06[6])->(state_derivative[12]) SXFunction)

In [ ]:
model.state_space.state_jacobian

Function(state_jacobian:(q[6],v[6],tau[3],link06[6])->(state_jacobian[12x12,78nz]) SXFunction)

In [ ]:
model.body("link06").contact.add_cone(mu=0.5, X=0.05, Y=0.02)

In [ ]:
wrench_cone = model.body("link06").contact.cone.full()

wrench_cone

Function(nonlin_wrench_cone:(force[6])->(constraint[6]) SXFunction)

In [ ]:
model.body("link06").contact.cone.linear()

DM(
[[-1, 0, -0.5, 0, 0, 0], 
 [1, 0, -0.5, 0, 0, 0], 
 [0, -1, -0.5, 0, 0, 0], 
 [0, 1, -0.5, 0, 0, 0], 
 [0, 0, -0.02, -1, 0, 0], 
 [0, 0, -0.02, 1, 0, 0], 
 [0, 0, -0.05, 0, -1, 0], 
 [0, 0, -0.05, 0, 1, 0], 
 [-0.02, -0.05, -0.035, 0.5, 0.5, -1], 
 [-0.02, 0.05, -0.035, 0.5, -0.5, -1], 
 [0.02, -0.05, -0.035, -0.5, 0.5, -1], 
 [0.02, 0.05, -0.035, -0.5, -0.5, -1], 
 [0.02, 0.05, -0.035, 0.5, 0.5, 1], 
 [0.02, -0.05, -0.035, 0.5, -0.5, 1], 
 [-0.02, 0.05, -0.035, -0.5, 0.5, 1], 
 [-0.02, -0.05, -0.035, -0.5, -0.5, 1]])

In [ ]:
model.add_body({"ee": "link06"})
model.bodies

dict_keys(['link06', 'ee'])

One can add bodies on the initialization stage based on following syntax:

In [ ]:
# Symbolic(z1_description.URDF_PATH, bodies_names={'shoulder':'link03', 'ee':'link06'})

The `bodies_names` arguments can be listof body names present in urdf, however for increased readability we suggest to use the dictionary as shown above.